<h1>Índice<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Carga-de-documentos" data-toc-modified-id="Carga-de-documentos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Carga de documentos</a></span><ul class="toc-item"><li><span><a href="#Preprocesado" data-toc-modified-id="Preprocesado-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preprocesado</a></span><ul class="toc-item"><li><span><a href="#Bigramas" data-toc-modified-id="Bigramas-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Bigramas</a></span></li></ul></li></ul></li><li><span><a href="#Glosario" data-toc-modified-id="Glosario-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Glosario</a></span><ul class="toc-item"><li><span><a href="#Extracción-de-keywords" data-toc-modified-id="Extracción-de-keywords-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Extracción de keywords</a></span><ul class="toc-item"><li><span><a href="#Extracción-propia" data-toc-modified-id="Extracción-propia-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Extracción propia</a></span></li><li><span><a href="#Gensim" data-toc-modified-id="Gensim-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Gensim</a></span></li><li><span><a href="#Kmeans" data-toc-modified-id="Kmeans-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Kmeans</a></span></li></ul></li><li><span><a href="#Formación-del-glosario" data-toc-modified-id="Formación-del-glosario-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Formación del glosario</a></span><ul class="toc-item"><li><span><a href="#Automatizado" data-toc-modified-id="Automatizado-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Automatizado</a></span></li></ul></li></ul></li><li><span><a href="#Clasificador" data-toc-modified-id="Clasificador-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Clasificador</a></span><ul class="toc-item"><li><span><a href="#Carga-de-glosarios" data-toc-modified-id="Carga-de-glosarios-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Carga de glosarios</a></span></li><li><span><a href="#Bigramas-de-test-data" data-toc-modified-id="Bigramas-de-test-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Bigramas de test data</a></span></li><li><span><a href="#Modelos" data-toc-modified-id="Modelos-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Modelos</a></span><ul class="toc-item"><li><span><a href="#TFIDF" data-toc-modified-id="TFIDF-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>TFIDF</a></span></li><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Word2Vec</a></span></li><li><span><a href="#Naive-Bayes" data-toc-modified-id="Naive-Bayes-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Naive Bayes</a></span><ul class="toc-item"><li><span><a href="#TFIDF" data-toc-modified-id="TFIDF-3.3.3.1"><span class="toc-item-num">3.3.3.1&nbsp;&nbsp;</span>TFIDF</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Clasificación-de-documentos" data-toc-modified-id="Clasificación-de-documentos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clasificación de documentos</a></span><ul class="toc-item"><li><span><a href="#Funciones-auxiliares" data-toc-modified-id="Funciones-auxiliares-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Funciones auxiliares</a></span></li><li><span><a href="#Clasificación" data-toc-modified-id="Clasificación-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Clasificación</a></span></li></ul></li><li><span><a href="#Evaluación-de-modelos" data-toc-modified-id="Evaluación-de-modelos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluación de modelos</a></span><ul class="toc-item"><li><span><a href="#Funciones-auxiliares" data-toc-modified-id="Funciones-auxiliares-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Funciones auxiliares</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import glob
import re
import unicodedata

# NLTK
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.util import ngrams

# Joblib
from joblib import Parallel, delayed

# Gensim
from gensim import corpora
from gensim import models
from gensim.summarization import keywords
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.similarities import Similarity

# Operatos
from operator import itemgetter

# Spacy
import spacy
from spacy_spanish_lemmatizer import SpacyCustomLemmatizer

# Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# statistics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report

# utils
from time import sleep
import os

# Carga de documentos

In [2]:
path_health = "../documents/health"
path_politics = "../documents/politics"
path_sports = "../documents/sports"
path_technology = "../documents/technology"
path_documents = "../documents"
path_stopwords = "../documents/stopwords.txt"

In [3]:
def load_document(path):
    return path.split("\\")[-1], open(path,encoding='utf-8').read(), path.split("\\")[-2]

In [4]:
documents = Parallel(n_jobs = -1)(delayed(load_document)(path) for path in glob.glob(path_documents+"/*/*.txt"))
documents = pd.DataFrame(documents, columns=["doc_name", "text", "class"])
documents['text'] = documents['text']

In [5]:
documents.head()

,doc_name,text,class
0,health_1.txt,Aceptémoslo de una vez: perder peso de manera ...,health
1,health_10.txt,"Sin tiempo para hacer recuento de daños, irrum...",health
2,health_11.txt,Mucha gente intenta mostrar en las redes socia...,health
3,health_12.txt,Una faceta clave en la frenética lucha global ...,health
4,health_13.txt,La curva de contagios de coronavirus se mantie...,health


## Preprocesado

In [6]:
REPLACE_NO_SPACE = re.compile("(\&)|(\%)|(\$)|(\€)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\⁰)|(\•)|(\\')")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "
    
#nlp = spacy.load("es")
lemmatizer = SpacyCustomLemmatizer()

def load_stopwords(path):
    return [line.strip() for line in open(path_stopwords, encoding = "utf-8").readlines()]

STOP_WORDS = set(load_stopwords(path_stopwords))

def delete_stop_words(doc):
    tokens = wordpunct_tokenize(doc)
    clean = [token for token in tokens if token not in STOP_WORDS and len(token) > 2]
    return clean

def preprocess_document(document):
    document = REPLACE_NO_SPACE.sub(NO_SPACE, document.lower())
    document = REPLACE_WITH_SPACE.sub(SPACE, document)
    # tokens = wordpunct_tokenize(document)
    # tokens = delete_proper_nouns(tokens)
    return document

def lemmatize(tokens):
    tokens = nlp(" ".join(tokens))
    return [token.lemma_ for token in tokens]


# TODO: REVISAR ESTO

def delete_proper_nouns(tokens):
    # Tag the tokens with their type - ie are they nouns or not
    lTokens = pos_tag(tokens)
    # find all the proper nouns and print them out
    lTagDict = findtags('NNP', lTokens)
    return [token.lower() for token in tokens if token not in lTagDict]
    
def findtags(tag_prefix, tagged_text):
    """
    Find tokens matching the specified tag_prefix
    """
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    print(cfd.conditions())
    return [list(cfd[tag].keys()) for tag in cfd.conditions()][0]

In [ ]:
documents["preprocesado"] = documents["text"].apply(lambda x: preprocess_document(x))
documents["tokens"] = documents["preprocesado"].apply(lambda x: delete_stop_words(x))
# documents["lematizado"] = documents["preprocesado"].apply(lambda x: lemmatize(x))
documents.head()

In [ ]:
# Training
train_health = documents[documents["class"] == "health"].iloc[:15]
train_politics = documents[documents["class"] == "politics"].iloc[:15]
train_sports = documents[documents["class"] == "sports"].iloc[:15]
train_technology = documents[documents["class"] == "technology"].iloc[:15]

train_data = pd.concat([train_health, train_politics, train_sports, train_technology])
print(f"Training data ==> {len(train_data)} documents")

In [ ]:
# Testing
test_health = documents[documents["class"] == "health"].iloc[15:]
test_politics = documents[documents["class"] == "politics"].iloc[15:]
test_sports = documents[documents["class"] == "sports"].iloc[15:]
test_technology = documents[documents["class"] == "technology"].iloc[15:]

test_data = pd.concat([test_health, test_politics, test_sports, test_technology])
test_data.reset_index(inplace = True)
print(f"Testing data ==> {len(test_data)} documents")

### Bigramas

In [ ]:
def get_bigrams(documents, threshold):
    token_ = [doc.split(" ") for doc in documents]
    bigram = Phrases(token_, min_count=1, threshold=threshold, delimiter=b' ')
    bigram_phraser = Phraser(bigram)
    bigram_token = []
    for sent in token_:
        for bigram in bigram_phraser[sent]:
            if len(bigram.split(" ")) > 1: # comprobamos que realmente es un bigrama
                bigram_token.append(bigram) 
    return bigram_token
           
def check_bigram(x, bigrams):
    if x.find("jamón serrano") != -1 or x.find("jamón") != -1:
        print(x)
    return [bigram for bigram in bigrams if x.find(bigram) != -1]

In [ ]:
bigrams_sports = get_bigrams(train_sports["preprocesado"].values, 50)
bigrams_health = get_bigrams(train_health["preprocesado"].values, 50)
bigrams_politics = get_bigrams(train_politics["preprocesado"].values, 50)
bigrams_technology = get_bigrams(train_technology["preprocesado"].values, 50)
bigrams = get_bigrams(train_data["preprocesado"].values, 50)


train_sports["bigrams"] = train_sports["preprocesado"].apply(lambda x: check_bigram(x, bigrams_sports))
train_health["bigrams"] = train_health["preprocesado"].apply(lambda x: check_bigram(x, bigrams_health))
train_politics["bigrams"] = train_politics["preprocesado"].apply(lambda x: check_bigram(x, bigrams_politics))
train_technology["bigrams"] = train_technology["preprocesado"].apply(lambda x: check_bigram(x, bigrams_technology))

train_sports["tokens + bigrams"] = train_sports["tokens"] + train_sports["bigrams"]
train_health["tokens + bigrams"] = train_health["tokens"] + train_health["bigrams"]
train_politics["tokens + bigrams"] = train_politics["tokens"] + train_politics["bigrams"]
train_technology["tokens + bigrams"] = train_technology["tokens"] + train_technology["bigrams"]

train_data["bigrams"] = train_data["preprocesado"].apply(lambda x: check_bigram(x, bigrams))
train_data["tokens + bigrams"] = train_data["tokens"] + train_data["bigrams"]

# Glosario

## Extracción de keywords

### Extracción propia

In [ ]:
stopwords_dir = "../documents/stopwords.txt"

In [ ]:
def get_k_tfidf_keywords(df, k):
    tokens = df["tokens + bigrams"].values
    dictionary = corpora.Dictionary(tokens)
    bow = [dictionary.doc2bow(doc) for doc in tokens]
    tfidf = models.TfidfModel(bow)
    bow_tfidf = tfidf[bow]
    tfidf_dic = {dictionary.get(id): value for doc in bow_tfidf for id, value in doc}
    tfidf_list = [k for k, v in sorted(tfidf_dic.items(), key=lambda item: item[1], reverse = True)]
    return tfidf_list[:k]

In [ ]:
keywords_tfidf_health = get_k_tfidf_keywords(train_health, 250)
keywords_tfidf_politics = get_k_tfidf_keywords(train_politics, 250)
keywords_tfidf_sports = get_k_tfidf_keywords(train_sports, 250)
keywords_tfidf_technology = get_k_tfidf_keywords(train_technology, 250)

In [ ]:
def remove_duplicates(d1, d2, d3, d4):

    i1 = set(d1) & set(d2)
    i2 = set(d1) & set(d3)
    i3 = set(d1) & set(d4)
    i4 = set(d2) & set(d3)
    i5 = set(d3) & set(d4)
    
    deleted = set(list(i1.union(i2,i3,i4,i5)))
    for key in deleted:
        try:
            d1.remove(key)
        except:
            print(f"D1 no tiene {key}")
        try:
            d2.remove(key)
        except:
            print(f"D2 no tiene {key}")
        try:
            d3.remove(key)
        except:
            print(f"D3 no tiene {key}")
        try:
            d4.remove(key)
        except:
            print(f"D4 no tiene {key}")
            
    return d1, d2, d3, d4

In [ ]:
keywords_tfidf_health, keywords_tfidf_politics, keywords_tfidf_sports, keywords_tfidf_technology = remove_duplicates(
    keywords_tfidf_health, keywords_tfidf_politics, keywords_tfidf_sports, keywords_tfidf_technology)

### Gensim

In [ ]:
def get_k_gensim_keywords(data, k):
    data = data.copy()
    data["joined"] = data["tokens + bigrams"].apply(lambda x: " ".join(x))
    data['joined'] = data.joined.astype(str)
    data = " ".join(data["joined"].values)
    return [key[0] for key in keywords(data, scores=True, words=k, pos_filter=('NNP', 'JJ', "NNPS", "VB"))]

In [ ]:
keywords_gensim_health = get_k_gensim_keywords(train_health, 250)
keywords_gensim_politics = get_k_gensim_keywords(train_politics, 250)
keywords_gensim_sports = get_k_gensim_keywords(train_sports, 250)
keywords_gensim_technology = get_k_gensim_keywords(train_technology, 250)

keywords_gensim_health, keywords_k_gensim_politics, keywords_k_gensim_sports, keywords_k_gensim_technology = remove_duplicates(
    keywords_gensim_health, keywords_gensim_politics, keywords_gensim_sports, keywords_gensim_technology)

### Kmeans

In [ ]:
def get_k_kmeans_keywords(data, k):
    data = data.copy()
    data["joined"] = data["tokens"].apply(lambda x: " ".join(x))
    k_means_data = data["joined"].values
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    X = vectorizer.fit_transform(k_means_data)
    
    model = KMeans(n_clusters=4, init='k-means++', max_iter=1000, n_init=1, random_state = 5, algorithm="full")
    model.fit(X)
    
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    keywords_kmeans_politics = [terms[ind] for ind in order_centroids[0, :k]]
    keywords_kmeans_technology = [terms[ind] for ind in order_centroids[1, :k]]
    keywords_kmeans_sports = [terms[ind] for ind in order_centroids[2, :k]]
    keywords_kmeans_health = [terms[ind] for ind in order_centroids[3, :k]]
    
    return keywords_kmeans_politics, keywords_kmeans_health, keywords_kmeans_sports, keywords_kmeans_technology

In [ ]:
keywords_kmeans_politics, keywords_kmeans_health, keywords_kmeans_sports, keywords_kmeans_technology = get_k_kmeans_keywords(
    train_data, 200)
print(len(keywords_kmeans_politics), len(keywords_kmeans_health),
      len(keywords_kmeans_sports), len(keywords_kmeans_technology))
keywords_kmeans_politics, keywords_kmeans_health, keywords_kmeans_sports, keywÇords_kmeans_technology = remove_duplicates(
    keywords_kmeans_politics, keywords_kmeans_health, keywords_kmeans_sports, keywords_kmeans_technology)
print(len(keywords_kmeans_politics), len(keywords_kmeans_health),
      len(keywords_kmeans_sports), len(keywords_kmeans_technology))

## Formación del glosario

### Automatizado

In [ ]:
def check_relevant_keywords(d1, d2, d3):
    i1 = set(d1) & set(d2)
    i2 = set(d1) & set(d3)
    i3 = set(d2) & set(d3)
    
    deleted = set(list(i1.union(i2).union(i3)))
    return deleted

In [ ]:
relevant_keywords_politics = check_relevant_keywords(keywords_kmeans_politics, keywords_gensim_politics, keywords_tfidf_politics)
relevant_keywords_health = check_relevant_keywords(keywords_kmeans_health, keywords_gensim_health, keywords_tfidf_health)
relevant_keywords_sports = check_relevant_keywords(keywords_kmeans_sports, keywords_gensim_sports, keywords_tfidf_sports)
relevant_keywords_technology = check_relevant_keywords(keywords_kmeans_technology, keywords_gensim_technology, keywords_tfidf_technology)

print("Politics ==> ", relevant_keywords_politics)
print("Sports ==> ", relevant_keywords_sports)
print("Health ==> ", relevant_keywords_health)
print("Technology ==> ", relevant_keywords_technology)

# Clasificador

## Carga de glosarios

In [ ]:
path_keys_health = "../keywords/keys_health.txt"
path_keys_sports = "../keywords/keys_sports.txt"
path_keys_politics = "../keywords/keys_politics.txt"
path_keys_technology = "../keywords/keys_technology.txt"

In [ ]:
keys_health = [key.strip() for key in open(path_keys_health, encoding="utf-8").readlines()]
keys_sports = [key.strip() for key in open(path_keys_sports, encoding="utf-8").readlines()]
keys_politics = [key.strip() for key in open(path_keys_politics, encoding="utf-8").readlines()]
keys_technology = [key.strip() for key in open(path_keys_technology, encoding="utf-8").readlines()]

keys_dic = {-1: "unknown", 0: "health", 1: "sports", 2: "politics", 3: "technology"}
inverted_keys_dic = {"unknown": -1, "health": 0, "sports": 1, "politics": 2, "technology": 3}

## Bigramas de test data

In [ ]:
test_data["bigrams"] = test_data["preprocesado"].apply(lambda x: check_bigram(x, bigrams))
test_data["tokens + bigrams"] = test_data["tokens"] + test_data["bigrams"]
test_data

## Modelos

### TFIDF

In [ ]:
glossaries = [keys_health, keys_sports, keys_politics, keys_technology]

In [ ]:
dictionary = corpora.Dictionary(glossary for glossary in glossaries)
dictionary.save('keys.dict')  # store the dictionary, for future reference

In [ ]:
class MyCorpus:
    
    def __init__(self, docs, dictionary):
        self.docs = docs
        self.dict = dictionary
        
    def __iter__(self):
        for doc in self.docs:
            # assume there's one document per line, tokens separated by whitespace
            yield self.dict.doc2bow(doc)

In [ ]:
bow = MyCorpus(glossaries, dictionary)
corpora.MmCorpus.serialize("keys.mm", bow, metadata=True)

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
index_temp = get_tmpfile("index")
index = Similarity(index_temp, bow, num_features=len(dictionary))  # create index
index.save("keys.index")

In [ ]:
model_tfidf = models.TfidfModel(bow,smartirs="lpu")

In [ ]:
def classify_document_tfidf(model, dictionary, bow, index, documents, i, verbose = False):
    """
    Given a specific document, computes the ranking of the classes and returns the current class, 
    the predicted class and the probabilities for each class.
    
    """
    document = documents.iloc[i]
    pq = document["tokens + bigrams"]
    vq = dictionary.doc2bow(pq)
    qtfidf = model[vq]
    sim = index[qtfidf]

    ranking = sorted(enumerate(sim), key=itemgetter(1), reverse=True)

    if verbose:
        print("Document ==> " + document["text"][:100])
        for doc, score in ranking:
            cat = keys_dic[doc]
            print(f"[{cat}] ==> %.3f" % round(score,3))
            
    
    return [i, get_info_document(document, ranking, sim)]

In [ ]:
def get_info_document(document, ranking, sim):
    """
    Given a ranking of classes, returns the current class, the predicted class and the probabilities for each class.
    
    """
    current_class = inverted_keys_dic[document["class"]]
    
    if np.sum(sim) == 0.0:
        predicted_class = -1
        probabilities = np.array([1/3, 1/3, 1/3])
    else:
        predicted_class = ranking[0][0]
        tfidf_scores = np.array(sim)
        probabilities = tfidf_scores / np.sum(tfidf_scores)
    
    return {"current_class": current_class, "predicted_class": predicted_class, 
            "probabilities": probabilities}

In [ ]:
classify_document_tfidf(model_tfidf, dictionary, bow, index, test_data, 110, verbose = True)

In [ ]:
def classify_tfidf(function, model, dictionary, bow, index, data):
    def classify(doc_i):
        return function(model, dictionary, bow, index, data, doc_i)
    return classify

### Word2Vec

In [ ]:
glossaries = [keys_health, keys_sports, keys_politics, keys_technology]
model_w2v = models.Word2Vec(sentences = glossaries, window = 5, workers = 12, min_count = 1, seed=50)

model_w2v.save("word2vec.model")

In [ ]:
def get_max_min(model):
    vocab = model.wv.vocab
    
    maxs = []
    mins = []
    
    for key in vocab:
        maxs.append(max(model.wv[key]))
        mins.append(min(model.wv[key]))
    return max(maxs), min(mins)

In [ ]:
MAXI, MINI = get_max_min(model_w2v)

In [ ]:
def get_embeddings_from_document(model, document):
    embeddings = []
    
    for word in document:
        if word in model.wv:
            embeddings.append(model.wv[word])
        else: # no está en el vocab
            embeddings.append(np.random.uniform(low = MINI, high = MAXI, size = 100))
    
    return np.mean(embeddings, axis = 0)

In [ ]:
glossaries_vector = [get_embeddings_from_document(model_w2v, glossary) for glossary in glossaries]

In [ ]:
def get_info_document_w2v(document, ranking, sim):
    """
    Given a ranking of classes, returns the current class, the predicted class and the probabilities for each class.
    
    """
    current_class = inverted_keys_dic[document["class"]]
    
    if np.count_nonzero(sim) == 0:
        predicted_class = -1
        probabilities = np.array([1/4, 1/4, 1/4])
    else:
        predicted_class = ranking[0][0]
        w2v_scores = np.array(sim, dtype = "float32")
        probabilities = (w2v_scores - w2v_scores.min()) / (w2v_scores.max() - w2v_scores.min()) 
        probabilities /= np.sum(probabilities)

        
    return {"current_class": current_class, "predicted_class": predicted_class, 
            "probabilities": probabilities}

In [ ]:
def classify_doc_w2v(glossaries_vector, model, documents, i, verbose = False):
    document = documents.iloc[i]
    doc_vector = get_embeddings_from_document(model, document["tokens + bigrams"])

    ranking = [[i, cosine_similarity(np.array(doc_vector).reshape(1,-1), np.array(glossary).reshape(1,-1)).item()] 
               for i, glossary in enumerate(glossaries_vector)]
    
    sim = [rank[1] for rank in ranking] 
    
    ranking.sort(key = lambda x: x[1], reverse = True)
    
    if verbose:
        print("Document ==> " + document["text"][:100])
        print("Scores:")
        for doc, score in ranking:
            cat = keys_dic[doc]
            print(f"[{cat}] ==> %.3f" % round(score,3))
    
    return [i, get_info_document_w2v(document, ranking, sim)]

In [ ]:
classify_doc_w2v(glossaries_vector, model_w2v, test_data, 118, verbose = True)

In [ ]:
def classify_w2v(function, glossaries_vector, model, data):
    def classify(doc_i):
        return function(glossaries_vector, model, data, doc_i)
    return classify

### Naive Bayes

In [ ]:
def classify_doc_bayes(classifier, documents, i, verbose = False):
    doc = documents.iloc[i]
    #doc = vectorizer.transform([document["preprocesado"]])
    
    y_pred = classifier.predict_proba([doc["preprocesado"]])[0]
    
    ranking = [[i,prob] for i,prob in enumerate(y_pred)]
    
    probabilities = y_pred
    
    ranking.sort(key = lambda x: x[1], reverse = True)
    
    current_class = inverted_keys_dic[doc["class"]]
    predicted_class = ranking[0][0]
    
    
    if verbose:
        print("Document ==> " + doc["text"][:100])
        print("Scores:")
        for doc, score in ranking:
            cat = keys_dic[doc]
            print(f"[{cat}] ==> %.3f" % round(score,3))
    
    return [i, {"current_class": current_class, "predicted_class": predicted_class, 
            "probabilities": probabilities}]

In [ ]:
def classify_bayes(function, clf, documents):
    def classify(doc_i):
        return function(clf, documents, doc_i)
    return classify

#### TFIDF

In [ ]:
glossaries_joined = [" ".join(gloss) for gloss in glossaries]

X_train = glossaries_joined
y_train = [0,1,2, 3]

clf = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,2))),
                     ('clf', MultinomialNB(alpha=1e-1))])

clf.fit(X_train, y_train)

In [ ]:
classify_doc_bayes(clf, test_data, 110, verbose = True)

# Clasificación de documentos

## Funciones auxiliares

In [ ]:
def classify_documents(test_data, classify):
    """
    Classifies the documents given a specific classification function.
    
    """
    test_data = test_data.copy()
    
    infos = [classify(i) for i in range(len(test_data))]
    data = fill_test_data(test_data, infos)
        
    return data 

In [ ]:
def fill_test_data(test_data, infos):
    """
    Auxiliary function to fill the dataframe with info about the classification.
    
    """
    data = test_data.copy()
    current_class = pd.Series([info[1]["current_class"] for info in infos])
    predicted_class = pd.Series([info[1]["predicted_class"] for info in infos])
    p_health = pd.Series([info[1]["probabilities"][0] for info in infos])
    p_sports = pd.Series([info[1]["probabilities"][1] for info in infos])
    p_politics = pd.Series([info[1]["probabilities"][2] for info in infos])

    data["current_class"] = current_class
    data["predicted_class"] = predicted_class
    data["p_health"] = p_health
    data["p_sports"] = p_sports
    data["p_politics"] = p_politics
    
    return data

In [ ]:
def get_filename(df):
    """
    Computes the filename for each document based on the performed classification.
    
    """
    confidence = "%.3f" % df["confidence"]
    current_class = df["class"]
    predicted_class = df["predicted_class_name"]
    correct = current_class == predicted_class
    name = df["doc_name"].split(".")[0]
    
    return f"../classification/{predicted_class}/{confidence}_{name}-{correct}-{current_class}-{predicted_class}.txt"

In [ ]:
def write_file(path, content):
    """
    Writes a file given its path and content.
    
    """
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding = "utf-8") as f:
        f.write(content)

In [ ]:
def move_files(data, tables = False):
    """
    Moves the files to their corresponding new directory after classification is done.
    
    """
    data = data.copy()
    classes = [[0, "p_health"], [1, "p_sports"], [2, "p_politics"]]

    for cl in classes:
        docs = data[data["current_class"] == cl[0]]
        docs = docs.sort_values(by=[cl[1]], ascending=False)
        docs["predicted_class_name"] = docs["predicted_class"].apply(lambda x : keys_dic[x])
        docs["confidence"] = docs[["p_health", "p_sports", "p_politics"]].max(axis=1)
        docs["file"] = docs.apply(lambda x: get_filename(x), axis=1)
        docs.apply(lambda row: write_file(row["file"], row["text"]), axis = 1)
        if tables:
            # tabla para la memoria
            docs = docs[["doc_name", "class", "p_health", "p_sports", "p_politics", "predicted_class_name"]]
            docs = docs.rename(columns = {"predicted_class_name": "predicted_class"})
            print(docs.to_latex(bold_rows = True, float_format="%.2f", column_format = "llllll", index=False))

In [ ]:
def execute(test_data, classification_function, move = True, tables = False):
    """
    General function that classifies the documents.
    
    """
    data = test_data.copy()
    print("############################################################")
    print("Starting document´s classification...")
    data = classify_documents(data, classification_function)
    sleep(1)
    print("Document´s classification done...")
    if move:
        print("-----------------------------------------------------------")
        sleep(1)
        print("Moving files to the correct directories...")
        move_files(data, tables)
        sleep(1)
        print("Files moved.")
    print("############################################################")
    
    return data

## Clasificación

In [ ]:
data = execute(test_data, classify_tfidf(classify_document_tfidf, model_tfidf, dictionary, bow, index, test_data))
data

# Evaluación de modelos

## Funciones auxiliares

In [ ]:
def evaluate_single_model(data, model, classify_function):
    """
    Function that evaluates the performance of a specific model.
    
    """
    data = data.copy()
    print("#######################################################")
    print("Evaluating "+ model + "...")
    print("-------------------------------------------------------")
    data = classify_documents(data, classify_function)
    
    y_true = data["current_class"]
    y_pred = data["predicted_class"]
    original = len(y_pred)
    y_pred = data[data["predicted_class"] != -1]["predicted_class"]
    unknown = original - len(y_pred)
    y_true = y_true.loc[y_pred.index]
        
    print(classification_report(y_true, y_pred, target_names=["health", "sports", "politics", "technology"]))
    print("Confusion matrix ==>")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(f"{unknown} documents couldn´t been classified.")

    #precisions = []
    #recalls = []

    #for i in range(len(cm[0])):
     #   name = keys_dic[i]
      #  print(f"Computing statistics about {name}:")
       # recall = cm[i,i] / np.sum(cm[i,:])
      #  precision = cm[i,i] / np.sum(cm[:,i])
      #  print(f"\tPrecision ==> {precision}")
      #  print(f"\tRecall ==> {recall}")

      #  precisions.append(precision)
      #  recalls.append(recall)

    #precisions = np.array(precisions)
    #recalls = np.array(recalls)
    #f1 = f1_score(y_true, y_pred, average = "macro")
    #accuracy = accuracy_score(y_true, y_pred)
    #print(f"Average precision ==> {precisions.mean()}")
    #print(f"Average recall ==> {recalls.mean()}")
    #print(f"F1-Score ==> {f1}")
    #print(f"Overall accuracy score ==> {accuracy}")
    print("-------------------------------------------------------")
    print("Model evaluated")
    print("#######################################################")

In [ ]:
evaluate_single_model(test_data, "tf-idf", classify_tfidf(classify_document_tfidf, model_tfidf, dictionary, bow, index, test_data))

In [ ]:
evaluate_single_model(test_data, "Word2Vec", classify_w2v(classify_doc_w2v, glossaries_vector, model_w2v, test_data))

In [ ]:
evaluate_single_model(test_data, "Multinomial NB (tfidf)", classify_bayes(classify_doc_bayes, clf, test_data))